# Configuracion de drive (para guardar los resultados)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Configuracion de programas

In [ ]:
#Miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -q python=3.7
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

#rdkit
!conda install -q -y -c conda-forge rdkit

#py3dmol
!conda install -y -q -c conda-forge py3dmol
import py3Dmol

#openbabel
!apt install openbabel
!conda install -q -y -c conda-forge openbabel

#pymol
!apt install pymol 

#smina
!wget https://downloads.sourceforge.net/project/smina/smina.static
!chmod +x smina.static

#gnina
!wget https://github.com/gnina/gnina/releases/download/v1.0.1/gnina
!chmod +x gnina

#plip
!conda install -y -q -c conda-forge plip


--2021-06-18 15:16:55--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2021-06-18 15:16:55--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94235922 (90M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  89.87M   154MB/s    in 0.6s    

2021-06-18 15:16:55 (154 MB/s) - ‘Miniconda3-latest-Linux

# Scripts de analisis

In [ ]:
# Calcular coordenadas de sitio de union
import numpy as np
def get_center(ligando):
  atoms = np.array([[i[30:38], i[38:46], i[46:54]] for i in open(ligando)], dtype='float32')
  x, y, z = atoms.sum(axis=0)/len(atoms)
  print('%.3f, %.3f, %.3f'%(x, y, z))


# visualizacion de modelos
from ipywidgets import interact,fixed,IntSlider
import ipywidgets
from openbabel import pybel

def MolTo3DView(mol):

    mol.write('sdf','mol.sdf',overwrite=True)
    v = py3Dmol.view()
    v.addModel(open('receptor.pdb').read())
    v.setStyle({'cartoon':{},'stick':{'radius':.1}})
    v.addModel(open('control.pdb').read())
    v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
    v.addModel(open('mol.sdf').read())
    v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
    v.zoomTo({'model':2})
    return v

# visualizacion de interacciones
class Atom(dict):
    def __init__(self, line):
        self["type"] = line[0:6].strip()
        self["idx"] = line[6:11].strip()
        self["name"] = line[12:16].strip()
        self["resname"] = line[17:20].strip()
        self["chain"] = line[21:22].strip()
        self["resid"] = int(int(line[22:26]))
        self["x"] = float(line[30:38])
        self["y"] = float(line[38:46])
        self["z"] = float(line[46:54])
        self["sym"] = line[76:78].strip()

    def __str__(self):
        line = list(" " * 80)
        
        line[0:6] = self["type"].ljust(6)
        line[6:11] = self["idx"].ljust(5)
        line[12:16] = self["name"].ljust(4)
        line[17:20] = self["resname"].ljust(3)
        line[22:26] = str(self["resid"]).ljust(4)
        line[30:38] = str(self["x"]).rjust(8)
        line[38:46] = str(self["y"]).rjust(8)
        line[46:54] = str(self["z"]).rjust(8)
        line[76:78] = self["sym"].rjust(2)
        return "".join(line) + "\n"

class Molecule(list):
    def __init__(self, file):
        for line in file:
            if line.startswith(("ATOM", "HETATM")):
                self.append(Atom(line))
            
    def __str__(self):
        outstr = ""
        for at in self:
            outstr += str(at)
            
        return outstr

import xml.etree.ElementTree as ET


def getresidues(report):
    root = ET.parse(report).getroot()
    residues = {}
    for bindingsite in root.iter('bindingsite'):
        if bindingsite.attrib['has_interactions']:
            for bs_residue in bindingsite.find('bs_residues'):
                if bs_residue.attrib['contact'] == 'True':
                    residues[bs_residue.text] = bs_residue.attrib['aa']
    return residues


def getligand(report):
    root = ET.parse(report).getroot()
    residues = {}
    for bindingsite in root.iter('bindingsite'):
        if bindingsite.attrib['has_interactions']:
            return bindingsite.find('identifiers/hetid').text


def getinteractions(report):
    root = ET.parse(report).getroot()
    inter_list = []
    for bindingsite in root.iter('bindingsite'):
        if bindingsite.attrib['has_interactions']:
            for interactions in bindingsite.iter('interactions'):
                hydrogen_bonds = interactions.findall(
                    'hydrogen_bonds/hydrogen_bond')
                inter_list += get_interaction(
                    hydrogen_bonds, 'hbond')

                hydrophobic_interactions = interactions.findall(
                    'hydrophobic_interactions/hydrophobic_interaction')
                inter_list += get_interaction(
                    hydrophobic_interactions, 'hphobic')

                salt_bridges = interactions.findall(
                    'salt_bridges/salt_bridge')
                inter_list += get_interaction(
                    salt_bridges, 'sbridge')

                pi_stacks = interactions.findall('pi_stacks/pi_stack')
                inter_list += get_interaction(
                    pi_stacks, 'pistack')

                pi_cation_interactions = interactions.findall(
                    'pi_cation_interactions/pi_cation_interaction')
                inter_list += get_interaction(
                    pi_cation_interactions, 'pication')

                halogen_bonds = interactions.findall(
                     'halogen_bonds/halogen_bonds')
                
                inter_list += get_interaction(
                    halogen_bonds, 'halogen')

    return inter_list


def get_interaction(interaction, inter_type):
    interactions = []
    for i in interaction:
        interactions.append({'inter_type': inter_type,
                 'resnr': i.findall('resnr')[0].text,
                 'restype': i.findall('restype')[0].text,
                 'reschain': i.findall('reschain')[0].text,
                 'resnr_lig': i.findall('resnr_lig')[0].text,
                 'restype_lig': i.findall('restype_lig')[0].text,
                 'reschain_lig': i.findall('reschain_lig')[0].text,
                 'ligcoo': [float(a.text) for a in i.findall('ligcoo')[0]],
                 'protcoo': [float(a.text) for a in i.findall('protcoo')[0]]})
    return edit_interaction(interactions)

def edit_interaction(interactions):
  for i in interactions:
    if i['inter_type'] == 'hbond':
      i['dashed'] = False
      i['color'] = 'blue'
    elif i['inter_type'] == 'hphobic':
        i['dashed'] =  True
        i['color'] = 'gray'
    elif i['inter_type'] == 'sbridge':
        i['dashed'] =  True
        i['color'] = 'yellow'
        i['sphere'] = 'yellow'
    elif i['inter_type'] == 'pistack':
        i['dashed'] = True
        i['color'] = 'green'
        i['sphere'] = 'white'
    elif i['inter_type'] == 'pication':
        i['dashed'] = True
        i['color'] = 'orange'
        i['sphere'] = 'white'
    elif i['inter_type'] == 'halogen':
        i['dashed'] = False
        i['color'] = 'cyan'
      
  return interactions

def draw_interaction(interaction):
  view.addCylinder({
  'radius':.1,
  'start':{'x':interaction['ligcoo'][0],'y': interaction['ligcoo'][1],'z':interaction['ligcoo'][2]},
  'end':{'x':interaction['protcoo'][0],'y': interaction['protcoo'][1],'z':interaction['protcoo'][2]}, 
  'color':interaction['color'],
  'dashed': interaction['dashed'],
  'fromCap': 1,
  'toCap': 1})
  if 'sphere' in interaction:
    view.addSphere({
      'center':{'x':interaction['ligcoo'][0],'y': interaction['ligcoo'][1],'z':interaction['ligcoo'][2]},
      'radius':.2,
      'color':interaction['sphere']})
    view.addSphere({
      'center':{'x':interaction['protcoo'][0],'y': interaction['protcoo'][1],'z':interaction['protcoo'][2]}, 
      'radius':.2,
      'color':interaction['sphere']})

#etiquetas
label = lambda i: '%s%s(%s)'%(i['restype'].title(),i['resnr'],i['reschain'])




In [ ]:
%%bash
cat > calcular_interacciones.sh <<EOF
CreateComplex(){
        id=\$1
        prot=\$2
        echo \$id
        obabel \${id}_out.sdf -O \${id}_out.pdb -m
        for i in \$(ls \${id}_out*.pdb)
        do
                grep "ATOM\|HETATM" \$prot > complejo_\$i
                echo TER >> complejo_\$i
                grep "ATOM\|HETATM" \$i | sed 's/ATOM  /HETATM/g'>> complejo_\$i
                plip -f complejo_\$i -x --name \$i
                rm complejo_\$i
        done
        rm \${id}_out*.pdb
}
list=\$1
for line in \$(cat \$list)
do
  CreateComplex \$line receptor.pdb
done
EOF

In [ ]:
%%bash
cat > reports2csv.py <<EOF
import xml.etree.ElementTree as ET
import pandas as pd
from collections import Counter
import sys

aa = {'ASP': ['acidic', 'acyclic', 'charged', 'medium', 'negative', 'polar', 'surface'],
        'GLU': ['acidic', 'acyclic', 'charged', 'large', 'negative', 'polar', 'surface'],
        'ALA': ['acyclic', 'aliphatic', 'buried', 'hydrophobic', 'neutral', 'small'],
        'GLY': ['acyclic', 'aliphatic', 'neutral', 'small', 'surface'],
        'LEU': ['acyclic', 'aliphatic', 'buried', 'hydrophobic', 'large', 'neutral'],
        'SER': ['acyclic', 'neutral', 'polar', 'small', 'surface'],
        'VAL': ['acyclic', 'aliphatic', 'buried', 'hydrophobic', 'medium', 'neutral'],
        'THR': ['acyclic', 'medium', 'neutral', 'polar', 'surface'],
        'LYS': ['acyclic', 'basic', 'charged', 'large', 'positive', 'surface'],
        'ILE': ['acyclic', 'aliphatic', 'buried', 'hydrophobic', 'large', 'neutral', 'polar', 'surface'],
        'ASN': ['acyclic', 'medium', 'neutral', 'polar', 'surface'],
        'GLN': ['acyclic', 'large', 'neutral', 'polar', 'surface'],
        'CYS': ['acyclic', 'buried', 'medium', 'neutral', 'polar'],
        'MET': ['acyclic', 'buried', 'hydrophobic', 'large', 'neutral'],
        'HIS': ['aromatic', 'basic', 'charged', 'cyclic', 'large', 'neutral', 'polar', 'positive', 'surface'],
        'PHE': ['aromatic', 'buried', 'cyclic', 'hydrophobic', 'large', 'neutral'],
        'TRP': ['aromatic', 'buried', 'cyclic', 'hydrophobic', 'large', 'neutral'],
        'TYR': ['aromatic', 'cyclic', 'large', 'neutral', 'surface'],
        'ARG': ['basic', 'charged', 'large', 'polar', 'positive', 'surface'],
        'PRO': ['cyclic', 'hydrophobic', 'medium', 'neutral', 'surface']}

ii = {'hydrophobic_interaction': 'hphobic',
        'hydrogen_bond':'hbond',
        'water_bridge':'water',
        'salt_bridge':'sbridge',
        'pi_stack':'pistack',
        'pi_cation_interaction':'pication',
        'halogen_bond':'halogen',
        'metal_complex':'metal'}

def get_bsresidues(bs_residues):
    bsresidues = Counter()
    for element in bs_residues:
        if element.attrib['contact'] == "True":
            bsresidues.update(aa[element.attrib['aa']])
    return dict(bsresidues)

def get_ligproperties(lig_properties):
    ligproperties = {}
    for element in lig_properties:
        ligproperties[element.tag] = element.text

    return ligproperties

def get_interactions(_interactions):
    interactions = Counter()
    for element in _interactions:
        for subelement in element:
            intertype = ii[subelement.tag]
            resnr = subelement.findall('resnr')[0].text
            restype = subelement.findall('restype')[0].text
            reschain = subelement.findall('reschain')[0].text
            interactions.update([intertype + "_" + restype.title() + resnr + reschain])

    return dict(interactions)

def parse_bindingsite(bindingsite):
    for element in bindingsite:
        if element.tag == 'lig_properties':
            ligproperties = get_ligproperties(element)
        elif element.tag == 'bs_residues':
            bsresidues = get_bsresidues(element)
        elif element.tag == 'interactions':
            interactions = get_interactions(element)

    return ligproperties, bsresidues, interactions

def parse_report(xml):
    tree = ET.parse(xml)
    root = tree.getroot()
    for i in root:
        if i.tag == 'bindingsite':
            ligproperties, bsresidues, interactions  = parse_bindingsite(i)

    return {**ligproperties, **bsresidues, **interactions}

def main():
    infile = sys.argv[1] # reports list
    outfile = sys.argv[2] # out.csv

    rename = lambda name: name.split('.')[0].replace('out','')

    reports = {}
    for report in open(infile):
        report = report.strip()
        name = rename(report)
        reports[name] = parse_report(report)
    df = pd.DataFrame.from_dict(reports)
    df = df.transpose()
    df.to_csv(outfile)

main()
EOF

# Practica #2

## Preparacion

### Preparacion de receptor

In [ ]:
# Descargar la estructura PDB
!wget http://files.rcsb.org/download/6ZRU.pdb

# Extraer atomos de la proteína
!grep ATOM 6ZRU.pdb | awk -v FS='' '$22 == "A" {print $0}' > receptor.pdb

# Preparar estructura del receptor
!pymol receptor.pdb -c -d "remove resn HOH;h_add acceptors or donors;save receptor.pdb"

--2021-06-18 15:10:40--  http://files.rcsb.org/download/6ZRU.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.158.70
Connecting to files.rcsb.org (files.rcsb.org)|128.6.158.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘6ZRU.pdb’

6ZRU.pdb                [ <=>                ] 227.26K  --.-KB/s    in 0.03s   

2021-06-18 15:10:40 (7.95 MB/s) - ‘6ZRU.pdb’ saved [232713]

 PyMOL(TM) Molecular Graphics System, Version 2.5.0.
 Copyright (c) Schrodinger, LLC.
 All Rights Reserved.
 
    Created by Warren L. DeLano, Ph.D. 
 
    PyMOL is user-supported open-source software.  Although some versions
    are freely available, PyMOL is not in the public domain.
 
    If PyMOL is helpful in your work or study, then please volunteer 
    support for our ongoing efforts to create open and affordable scientific
    software by purchasing a PyMOL Maintenance and/or Support subscription.

    More information can be 

### Preparacion de ligando

In [ ]:
# Extraer los atomos del ligando
!grep -E HETATM.*U5G 6ZRU.pdb | awk -v FS='' '$22 == "A" {print $0}' > control.pdb
!wget https://files.rcsb.org/ligands/view/U5G_ideal.sdf
#!obabel -:'CC1([C@@H]2[C@H]1[C@H](N(C2)C(=O)[C@H](C(C)(C)C)NC(=O)NC(C)(C)C)C(=O)N[C@@H](CC3CCC3)[C@H](C(=O)N)O)C' --gen3D best -O ligando.pdb

--2021-06-18 15:10:41--  https://files.rcsb.org/ligands/view/U5G_ideal.sdf
Resolving files.rcsb.org (files.rcsb.org)... 128.6.158.70
Connecting to files.rcsb.org (files.rcsb.org)|128.6.158.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘U5G_ideal.sdf’

U5G_ideal.sdf           [ <=>                ]   8.10K  --.-KB/s    in 0s      

2021-06-18 15:10:41 (89.3 MB/s) - ‘U5G_ideal.sdf’ saved [8291]



## Acoplamiento molecular

In [ ]:
!./gnina -r receptor.pdb -l U5G_ideal.sdf --autobox_ligand control.pdb -o gnina_lig.sdf --cnn_scoring=none

              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.0.1 HEAD:aa41230   Built Mar 23 2021.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Commandline: ./gnina -r receptor.pdb -l U5G_ideal.sdf --autobox_ligand control.pdb -o gnina_lig.sdf --cnn_scoring=none
Using random seed: -752763284

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
**********************************

## Analisis de resultados

### RMSD

In [ ]:
# Calculo de RMSD
!obrms --firstonly control.pdb gnina_lig.sdf

### Visualizacion


In [ ]:
# Visualizacion de poses
confs = [s for s in  pybel.readfile('sdf', 'gnina_lig.sdf')]

def conf_viewer(idx):
    mol = confs[idx]
    return MolTo3DView(mol).show()

interact(conf_viewer, idx=IntSlider(min=0,max=9-1, step=1))

## Calculo de interacciones

### control

In [ ]:
# Construccion del complejo receptor-ligando
!cat receptor.pdb > control_complejo.pdb
!echo TER >> complejo.pdb
!grep HETATM ligando.pdb >> control_complejo.pdb

In [ ]:
!plip -f control_complejo.pdb -t --name control_interacciones

### gnina

In [ ]:
# Seleccion de modelo
!obabel gnina_lig.sdf -f1 -l1 -O gnina_lig_best.pdb 

# Construccion del complejo receptor-ligando
!grep "ATOM\|HETATM" receptor.pdb > gnina_complejo.pdb
!echo TER >> gnina_complejo.pdb
!grep HETATM gnina_lig_best.pdb >> gnina_complejo.pdb

In [ ]:
!plip -f gnina_complejo.pdb -tx --name gnina_interacciones

In [ ]:
interactions = [i for i in getinteractions('gnina_interacciones.xml') if i['restype_lig']=='UNL']
residues = [i['resnr'] for i in interactions]

with open('gnina_complejo_protonated.pdb') as ifile:
    mol = Molecule(ifile)

for at in mol:
    if str(at["resid"]) in residues:
        at["pymol"] = {"stick": {}}
    elif at["resname"] == 'UNL':
        sel = at
        at["pymol"] = {"stick": {'colorscheme': 'greenCarbon'}}

view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(str(mol))
for i, at in enumerate(mol):
  default = {"": {'color': 'white'}}
  view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))
  view.setColorByElement(at.get("pymol", default))
labels = []
for i in interactions:
  draw_interaction(i)
  if i['resnr'] not in labels:
    view.addLabel(label(i),
                  {'fontSize':15,
                   'position':
                   {'x':i['protcoo'][0], 'y':i['protcoo'][1], 'z':i['protcoo'][2]},
                   'screenOffset':{'x':0, 'y':0}})
    labels.append(i['resnr'])
view.zoomTo({'model': -1, 'resn': 'UNL'})
view.addModel(open('control.pdb').read())
view.setStyle({'model':-1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
view.show()

# Practica #3

## Acoplamiento multiple

### Preparacion de moleculas

#### Preparacion del receptor

In [ ]:
# Descargar la estructura PDB
!wget http://files.rcsb.org/download/7JKV.pdb

# Extraer atomos de la proteína
!grep ATOM 7JKV.pdb | awk -v FS='' '$22 == "A" {print $0}' > receptor.pdb

# Preparar estructura del receptor
!pymol receptor.pdb -c -d "remove resn HOH;h_add acceptors or donors;save receptor.pdb"

#### Preparacion del ligandos


In [ ]:
# Extraer los atomos del ligando control
!grep -E HETATM.*V7G 7JKV.pdb | awk -v FS='' '$22 == "A" {print $0}' > control.pdb

In [ ]:
%%bash
cat > ligandos.smi <<EOF
CCCC[C@@H](CO)NC(=O)[C@H](CC(C)C)NC(=O)OCc1ccccc1 Calpeptin
c1ccc(cc1)C(c2ccccc2)S(=O)CC(=O)NO Adrafinil
CCc1cc(c(cc1F)Oc2ccc(cc2F)C(=O)N)O MUT056399
CC(C)(C)C(=O)[C@@H](n1ccnc1)Oc2ccc(cc2)Cl Climbazole
CC1CN(CCO1)C(=O)CN2C=CC=N2 GRL-2420
EOF

In [ ]:
# Descargar la estructura del ligando
!obabel ligandos.smi --gen3d best -O ligandos.sdf

### Acoplamiento

In [ ]:
!./gnina -r receptor.pdb -l ligandos.sdf --autobox_ligand control.pdb -o gnina_out.sdf --cnn_scoring=none

### Analisis de resultados

In [ ]:
#Dividir archivos
inputfile = [m for m in pybel.readfile('sdf', 'gnina_out.sdf')]
ligandos = {}
for i in inputfile:
  if i.title in ligandos:
    ligandos[i.title].append(i)
  else:
    ligandos[i.title] = [i]

for ligando in ligandos:
  f = pybel.Outputfile('sdf', '%s_out.sdf'%ligando)
  for pose in ligandos[ligando]:
    f.write(pose)
  f.close()

#### Visualizacion


In [ ]:
# Visualizacion de poses
confs = [s for s in  pybel.readfile('sdf', 'Adrafinil_out.sdf')]

def conf_viewer(idx):
    mol = confs[idx]
    return MolTo3DView(mol).show()

interact(conf_viewer, idx=IntSlider(min=0,max=9-1, step=1))

#### Calculo de interacciones

#### control

In [ ]:
# Construccion del complejo receptor-ligando
!grep "ATOM\|HETATM" receptor.pdb > control_complejo.pdb
!echo TER >> control_complejo.pdb
!grep HETATM control.pdb >> control_complejo.pdb

In [ ]:
!plip -f control_complejo.pdb -tx --name control_interacciones

#### gnina

In [ ]:
# Seleccion de modelo
!obabel Adrafinil_out.sdf -f1 -l1 -O Adrafinil_best.pdb 

# Construccion del complejo receptor-ligando
!grep "ATOM\|HETATM" receptor.pdb > Adrafinil_complejo.pdb
!echo TER >> Adrafinil_complejo.pdb
!grep HETATM Adrafinil_best.pdb >> Adrafinil_complejo.pdb

In [ ]:
!plip -f Adrafinil_complejo.pdb -tx --name Adrafinil_interacciones

In [ ]:
interactions = [i for i in getinteractions('Adrafinil_interacciones.xml') if i['restype_lig']=='UNL']
residues = [i['resnr'] for i in interactions]

with open('Adrafinil_complejo_protonated.pdb') as ifile:
    mol = Molecule(ifile)

for at in mol:
    if str(at["resid"]) in residues:
        at["pymol"] = {"stick": {}}
    elif at["resname"] == 'UNL':
        sel = at
        at["pymol"] = {"stick": {'colorscheme': 'greenCarbon'}}

view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(str(mol))
for i, at in enumerate(mol):
  default = {"": {'color': 'white'}}
  view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))
  view.setColorByElement(at.get("pymol", default))
labels = []
for i in interactions:
  draw_interaction(i)
  if i['resnr'] not in labels:
    view.addLabel(label(i),
                  {'fontSize':15,
                   'position':
                   {'x':i['protcoo'][0], 'y':i['protcoo'][1], 'z':i['protcoo'][2]},
                   'screenOffset':{'x':0, 'y':0}})
    labels.append(i['resnr'])
view.zoomTo({'model': -1, 'resn': 'UNL'})
view.show()

In [ ]:
!cut -d' ' -f2 ligandos.smi > lista
!bash calcular_interacciones.sh lista

In [ ]:
!ls *pdb.xml > lista_xml.txt
!python3 reports2csv.py lista_xml.txt interacciones.csv